In [1]:
import pandas as pd
import numpy as np

#Load in raw excel data
EXP = pd.read_excel(r'MasterNuclei.xlsx', sheet_name = 'Exp') #experimental


ME2 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'DD-ME2_even-even_nuclei')
MEdelta = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'DD-MEdelta_even-even_nuclei')
PC1 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'DD-PC1_even-even_nuclei')
NL3S = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'NL3S_even-even_nuclei')


SKMS = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'SKMS_all_nuclei')
SKP = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'SKP_all_nuclei')
SLY4 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'SLY4_all_nuclei')
SV = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'SV-MIN_all_nuclei')
UNEDF0 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'UNEDF0_all_nuclei')
UNEDF1 = pd.read_excel (r'MasterNuclei.xlsx', sheet_name = 'UNEDF1_all_nuclei')

SKMS_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'SKMS_even-even_nuclei')
SKP_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'SKP_even-even_nuclei')
SLY4_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'SLY4_even-even_nuclei')
SV_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'SV-MIN_even-even_nuclei')
UNEDF0_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'UNEDF0_even-even_nuclei')
UNEDF1_even = pd.read_excel (r'All_Even-Even_Nuclei.xlsx', sheet_name = 'UNEDF1_even-even_nuclei')


#raw = tuple([EXP, ME2, MEdelta, PC1, NL3S, SKMS, SKP, SLY4, SV, UNEDF0, UNEDF1])

In [45]:

def BE(Z,N,df):
    return df[df['Z']==Z & df['N']==N]['Binding_Energy_(MeV)']

def OneNSE(Z,N,df):
    try:
        return BE(Z,N,df) - BE(Z,N-1,df)
    except:
        return None
    
def OnePSE(Z,N,df):
    try:
        return BE(Z,N,df) - BE(Z-1,N,df)
    except:
        return None

def TwoNSE(Z,N,df):
    try:
        return BE(Z,N,df) - BE(Z,N-2,df)
    except:
        return None

def TwoPSE(Z,N,df):
    try:
        return BE(Z,N,df) - BE(Z-2,N,df)
    except:
        return None

def AlphaSE(Z,N,df):
    try:
        return BE(Z,N,df) - BE(Z-2,N-2,df) - 28.3
    except:
        return None

def TwoPSGap(Z,N,df):
    try:
        return TwoPSE(Z,N,df) - TwoPSE(Z+2,N,df)
    except:
        return None

def TwoNSGap(Z,N,df):
    try:
        return TwoNSE(Z,N,df) - TwoNSE(Z,N+2,df)
    except:
        return None

def DoubleMDiff(Z,N,df):
    try:
        return (TwoPSE(Z,N,df) - TwoPSE(Z,N-2,df))/4
    except:
        return None

def N3PointOED(Z,N,df):
    try:
        return (-1)**N*(OneNSE(Z,N,df) - OneNSE(Z,N+1))/2
    except:
        return None

def P3PointOED(Z,N,df):
    try:
        return (-1)**Z*(OnePSE(Z,N,df) - OnePSE(Z+1,N))/2
    except:
        return None

def SNESplitting(Z,N,df):
    try:
        return (-1)**N*(OneNSE(Z,N,df) - OneNSE(Z,N+2,df))
    except:
        return None

def SPESplitting(Z,N,df):
    try:
        return (-1)**Z*(OnePSE(Z,N,df) - OnePSE(Z+2,N,df))
    except:
        return None

def WignerEC(Z,N,df):
    if Z!=N:
        return None
    try:
        return DoubleMDiff(Z,N,df)-(DoubleMDiff(Z-2,N,df) + DoubleMDiff(Z,N+2,df))/2
    except:
        return None

In [52]:
#Collect all raw model data into each individual df

covar = [ME2, MEdelta, PC1, NL3S]

skm = [SKMS, SKP, SLY4, SV, UNEDF0, UNEDF1]
skm_even = [SKMS_even, SKP_even, SLY4_even, SV_even, UNEDF0_even, UNEDF1_even]
skm_merged = []
for i in range(len(skm)):
    skm_merged.append(skm[i].combine_first(skm_even[i]))

raw = [EXP]

for c in covar:
    raw.append(c)
for s in skm_merged:
    raw.append(s)

In [55]:
filtered = []
for d in raw:
    # df = d.loc[:,['Z', 'N', 'Binding_Energy_(MeV)', ]]
    d['Binding_Energy_(MeV)'] = abs(d['Binding_Energy_(MeV)'])
    filtered.append(d)
filtered[3]

,Z,N,Equilibrium_Quad_Def,Binding_Energy_(MeV),Chemical_Potential_N_(MeV),Chemical_Potential_P_(MeV),Pairing_Energy_N_(MeV),Pairing_Energy_P_(MeV),Charge_Radius_(fm),Quad_Def_Beta2_N,Quad_Def_Beta2_P,Quad_Def_Beta4_N,Quad_Def_Beta4_P,Mass_Radius_N_(fm),Mass_Radius_P_(fm)
0,2,2,0.001,21.017,-16.4,-15.1,0.00,0.00,2.183,0.001,0.001,-0.001,-0.001,2.017,2.031
1,2,4,0.005,27.777,-2.0,-23.9,-6.20,0.00,2.136,0.008,0.000,0.000,0.000,2.567,1.981
2,2,6,0.000,30.309,-0.8,-30.8,-5.60,0.00,2.135,0.000,0.000,0.000,0.000,2.836,1.979
3,4,2,0.014,24.499,-24.7,-0.1,0.00,-6.16,2.757,0.001,0.020,0.000,0.004,1.979,2.639
4,4,4,0.000,46.425,-9.0,-6.9,-6.09,-6.06,2.498,0.000,0.000,0.000,0.000,2.344,2.366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756,116,176,-0.210,2077.402,-6.6,-2.1,-2.66,-7.21,6.310,-0.208,-0.212,0.075,0.077,6.398,6.259
1757,116,178,-0.230,2090.569,-6.1,-2.5,0.00,-7.05,6.333,-0.229,-0.232,0.085,0.087,6.429,6.283
1758,118,174,-0.180,2067.141,-7.2,-1.0,0.00,-7.66,6.300,-0.178,-0.182,0.054,0.056,6.369,6.249
1759,118,176,-0.210,2080.712,-7.0,-1.2,-2.71,-7.88,6.330,-0.208,-0.212,0.071,0.073,6.405,6.279


In [65]:
names = ['EXP', 'ME2', 'MEdelta', 'PC1', 'NL3S', 'SKMS', 'SKP', 'SLY4', 'SV', 'UNEDF0', 'UNEDF1']
qinput = ['BE', 'OneNSE', 'OnePSE', 'TwoNSE', 'TwoPSE', 'AlphaSE', 'TwoNSGap', 'TwoPSGap', 'DoubleMDiff', 'N3PointOED', 'P3PointOED', 'SNESplitting', 'SPESplitting', 'WignerEC', 'QDB2t']
master_df = pd.DataFrame(
    columns=['Model', 'Z', 'N', 'BE', 'Binding_Energy_(MeV)', 'Fermi_Energy_P_(MeV)', 
    'Fermi_Energy_N_(MeV)', 'Quad_Def_Beta2_P',	'Quad_Def_Beta2_N',	'Quad_Def_Beta2_total',	
    'Quad_Moment_Q2_P_(fm^2)', 'Quad_Moment_Q2_N_(fm^2)', 'Quad_Moment_Q2_total_(fm^2)', 
    'Pairing_gap_P_(MeV)', 'Pairing_gap_N_(MeV)', 'RMS_radius_P_(fm)', 'RMS_radius_N_(fm)', 
    'RMS_radius_total_(fm)', 'Neutron_skin_(fm)', 'Charge_Radius_(fm)', 'S_p_(MeV)', 
    'S_{2p}_(MeV)', 'S_n_(MeV)', 'S_{2n}_(MeV)', 'Q_{alpha}_(MeV)',
])

for m in range(len(filtered)):
    M = filtered[m]
    for rowi in range(len(M.index)):
        n = len(master_df.index)
        master_df.loc[n] = M.iloc[rowi,:]
        master_df.loc[n, 'Model'] = names[m]


In [64]:
# TEST
master_df = pd.DataFrame(
    columns=['Model', 'Z', 'N', 'Binding_Energy_(MeV)', 'Fermi_Energy_P_(MeV)', 
    'Fermi_Energy_N_(MeV)', 'Quad_Def_Beta2_P',	'Quad_Def_Beta2_N',	'Quad_Def_Beta2_total',	
    'Quad_Moment_Q2_P_(fm^2)', 'Quad_Moment_Q2_N_(fm^2)', 'Quad_Moment_Q2_total_(fm^2)', 
    'Pairing_gap_P_(MeV)', 'Pairing_gap_N_(MeV)', 'RMS_radius_P_(fm)', 'RMS_radius_N_(fm)', 
    'RMS_radius_total_(fm)', 'Neutron_skin_(fm)', 'Charge_Radius_(fm)', 'S_p_(MeV)', 
    'S_{2p}_(MeV)', 'S_n_(MeV)', 'S_{2n}_(MeV)', 'Q_{alpha}_(MeV)',
])
master_df.loc[0] = filtered[3].iloc[4,:]
master_df.loc[0, 'Model'] = "hey"
master_df

,Model,Z,N,Binding_Energy_(MeV),Fermi_Energy_P_(MeV),Fermi_Energy_N_(MeV),Quad_Def_Beta2_P,Quad_Def_Beta2_N,Quad_Def_Beta2_total,Quad_Moment_Q2_P_(fm^2),...,RMS_radius_P_(fm),RMS_radius_N_(fm),RMS_radius_total_(fm),Neutron_skin_(fm),Charge_Radius_(fm),S_p_(MeV),S_{2p}_(MeV),S_n_(MeV),S_{2n}_(MeV),Q_{alpha}_(MeV)
0,hey,4.0,4.0,46.425,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.498,NaN,NaN,NaN,NaN,NaN


In [19]:
print(master_df.head(),'\n', master_df.tail())

  Model  Z  N        BE
0   EXP  0  1  0.000000
1   EXP  1  0  0.000000
2   EXP  1  1 -2.224566
3   EXP  1  2 -8.481796
4   EXP  2  1 -7.718041 
         Model    Z    N           BE
57686  UNEDF1  120  289 -2408.792800
57687  UNEDF1  120  290 -2409.407277
57688  UNEDF1  120  292 -2409.876417
57689  UNEDF1  120  294 -2410.154072
57690  UNEDF1  120  296 -2410.250510


In [ ]:
master_df['One Neutron Seperation Energy'] = master_df['BE']